In [1]:
import re
import qiskit
import json
import time

from qiskit import transpile
import time
import os
import random

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
# from circuit_generator import construct_qcc_circuit
from utilities import generate_pauli_strings
from fast_generator import update_paulis
from qiskit.converters import circuit_to_dag

In [2]:
#first compare the UCCSD ansatz
test_paulis = generate_UCCSD_entanglers(10, 20)
test_params = [0.01 * i for i in range(len(test_paulis))]

start_time = time.time()
opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
end_time = time.time()
combined_time = end_time - start_time

In [3]:
opt_qiskit.count_ops()

OrderedDict([('rz', 36715), ('cx', 24302), ('sx', 23524), ('x', 907)])

In [4]:
for num_obs in [10, 50, 100, 500, 1000, 5000]:
    random_paulis = generate_pauli_strings(20, num_obs)
    start_time = time.time()
    new_obs = update_paulis(random_paulis, append_clifford_f2, parameters = False)
    end_time = time.time()
    compile_time = end_time - start_time
    print("num_obs", num_obs, "compile_time", compile_time)

num_obs 10 compile_time 0.051860809326171875
num_obs 50 compile_time 0.26828980445861816
num_obs 100 compile_time 0.45281982421875
num_obs 500 compile_time 2.3160064220428467
num_obs 1000 compile_time 4.539279460906982
num_obs 5000 compile_time 23.06069779396057


In [5]:
#then compare the compile time for MAXCUT 
with open("benchmarks/max_cut_paulis/max_cut_benchmark_regular_n20_e12_l1.json", "r") as file:
    test_paulis = json.load(file)
test_params = [0.01 * i for i in range(len(test_paulis))]

start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
end_time = time.time()
combined_time = end_time - start_time

In [6]:
opt_qiskit.count_ops()

OrderedDict([('cx', 171), ('rz', 156), ('sx', 20)])

In [7]:
def generate_binary_states(n, m):
    states = []
    for _ in range(m):
        state = ''.join(random.choice(['0', '1']) for _ in range(n))
        states.append(state)
    return states

In [8]:
def apply_cnot(binary_value, control_index, target_index):
    # Convert binary string to a list of characters for easy manipulation
    binary_list = list(binary_value)
    
    # Apply CNOT: If the control qubit is 1, flip the target qubit
    if binary_list[control_index] == '1':
        binary_list[target_index] = '0' if binary_list[target_index] == '1' else '1'
    
    # Convert list back to binary string
    return ''.join(binary_list)

In [9]:
def update_probabilities(states, circuit_dag):
    updated_states = []
    for state in states:
        for node in circuit_dag.topological_op_nodes():
            if node.name == 'cx':
                control_qubit = node.qargs[0]._index
                target_qubit = node.qargs[1]._index
                new_state = apply_cnot(state, control_qubit, target_qubit)
                updated_states.append(new_state)
    return updated_states


In [10]:
dag = circuit_to_dag(opt_qc.inverse())
for num_states in [10, 50, 100, 500, 1000, 5000]:
    random_states =  generate_binary_states(20, num_states)
    start_time = time.time()
    new_states = update_probabilities(random_states, dag)
    end_time = time.time()
    compile_time = end_time - start_time
    print("num_states", num_states, "compile_time", compile_time)

num_states 10 compile_time 0.0029921531677246094
num_states 50 compile_time 0.0169527530670166
num_states 100 compile_time 0.03191375732421875
num_states 500 compile_time 0.16050934791564941
num_states 1000 compile_time 0.3251302242279053
num_states 5000 compile_time 1.675201654434204
